In [1]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import random

In [2]:
class FullModel(torch.nn.Module):
    def __init__(self):
        super(FullModel, self).__init__()
        self.fc1 = torch.nn.Linear(1, 5)
        self.fc2 = torch.nn.Linear(5, 5)
        self.fc3 = torch.nn.Linear(5, 1)
        self.relu = torch.nn.ReLU()
        
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        return x

In [3]:
def func(x):
    return np.power(x, 1) * np.power(np.sin(1.0 / 3.0 * x), 2)

def avg_l2_diff(y1, y2):
    return np.average(np.power(y1-y2, 2))

In [4]:
def train(model, optimizer, criterion, epoch):
    model.train()
    
    running_loss  = torch.tensor(0.0)

    for batch_idx, current_batch in enumerate(train_loader):     
        inp, current_batch_y = current_batch[0],        current_batch[1]

        optimizer.zero_grad()
        output = model(inp)
        gndtruth = current_batch_y

        loss = criterion(output, gndtruth)
        loss.backward()
        optimizer.step()
        running_loss  += loss.item()

    running_loss  = running_loss  / len(train_loader)
    
    if epoch % 50 == 0:
        print("Epoch: {}, Average loss: {:15.8f}".format(epoch, running_loss))

In [5]:
l2_diff_tot = []
for seed in np.arange(1):
    torch.manual_seed(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.use_deterministic_algorithms(True)
    
    model = FullModel()
    #x_train = np.arange(-2.4, 3.2, 0.8)
    x_train = np.arange(-2.4, 2.8, 0.4)
    #x_train = np.arange(-2.4, 2.6, 0.2)
    #x_train = np.arange(-2.4, 2.5, 0.1)
    #x_train = np.arange(-2.4, 2.45, 0.05)

    y_train = func(x_train)
    x_train_torch = torch.from_numpy(x_train).float()
    y_train_torch = torch.from_numpy(y_train).float()
    train_dataset = torch.utils.data.TensorDataset(x_train_torch, y_train_torch)
    train_loader = torch.utils.data.DataLoader(
        train_dataset, shuffle=True)
    optimizer = torch.optim.SGD(model.parameters(), lr=0.01, weight_decay=0.005)
    criterion = torch.nn.MSELoss()
    
    for epoch in range(1, 3000):
        train(model, optimizer, criterion, epoch)
        
    x_pred_np = np.arange(-2.5, 2.5, 0.0001).reshape(-1, 1)
    x_pred = torch.from_numpy(x_pred_np).float()
    y_true = func(x_pred_np)
    y_pred = model(x_pred)
    x_np = x_pred.numpy().reshape(-1)
    y_pred_np = y_pred.detach().numpy().reshape(-1)
    y_true_np = y_true.reshape(-1)
    l2_diff = avg_l2_diff(y_pred_np, y_true_np).astype(float)
    l2_diff_tot.append(l2_diff)

Epoch: 50, Average loss:      0.07526521
Epoch: 100, Average loss:      0.05593012
Epoch: 150, Average loss:      0.03763933
Epoch: 200, Average loss:      0.02691914
Epoch: 250, Average loss:      0.02141721
Epoch: 300, Average loss:      0.01504277
Epoch: 350, Average loss:      0.01090206
Epoch: 400, Average loss:      0.00895169
Epoch: 450, Average loss:      0.00676315
Epoch: 500, Average loss:      0.00547076
Epoch: 550, Average loss:      0.00443839
Epoch: 600, Average loss:      0.00425628
Epoch: 650, Average loss:      0.00449957
Epoch: 700, Average loss:      0.00367146
Epoch: 750, Average loss:      0.00391440
Epoch: 800, Average loss:      0.00336614
Epoch: 850, Average loss:      0.00394943
Epoch: 900, Average loss:      0.00340245
Epoch: 950, Average loss:      0.00382954
Epoch: 1000, Average loss:      0.00383634
Epoch: 1050, Average loss:      0.00382960
Epoch: 1100, Average loss:      0.00404081
Epoch: 1150, Average loss:      0.00366357
Epoch: 1200, Average loss:     

In [6]:
print(l2_diff_tot)

[0.0029204877979537936]
